In [ ]:
from utils.model import get_model, get_vocoder
import easydict
import torch
import yaml

device = torch.device('cpu')

args = easydict.EasyDict()
args['restore_step'] = 7000
args['preprocess_config'] = './config/kss/preprocess.yaml'
args['model_config'] = './config/kss/model.yaml'
args['train_config'] = './config/kss/train.yaml'

preprocess_config = yaml.load(open(args.preprocess_config, "r"), Loader=yaml.FullLoader)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)

model = get_model(args, configs, device, False)
vocoder = get_vocoder(model_config, device)

In [ ]:
from utils.tools import to_device_inference, intersperse
from text import text_to_sequence
import json


def get_speaker_ids(path):
    with open(path, 'r', encoding='utf8') as j:
        speaker_ids = json.loads(j.read())
    return speaker_ids

def get_speaker_id(speaker, speaker_ids):
    return speaker_ids[speaker]

def get_text(sentence, preprocess_config):
    sequence = text_to_sequence(sentence)
    if preprocess_config["preprocessing"]["text"]["use_intersperse"]:
        sequence = intersperse(sequence, 0)
    return sequence

def get_pairs(speaker:str, sentence:str, speaker_ids:dict, preprocess_config:dict):
    speaker_id = torch.LongTensor([get_speaker_id('kss', speaker_ids, )]).long()
    sequence = torch.LongTensor(get_text(sentence, preprocess_config)).unsqueeze(0).long()
    src_lens = torch.LongTensor([sequence.size(0)]).long()
    max_src_len = src_lens.max().long()
    return (speaker_id, sequence, src_lens, max_src_len)

In [ ]:
speaker_ids = get_speaker_ids('./preprocessed_data/kss/speakers.json')
speaker = 'kss'
sentence = '안녕하세요? 만나서 반갑습니다.'

pairs = to_device_inference(
    get_pairs(speaker, sentence, speaker_ids, preprocess_config), 
    device
)

controls = {
    'p_control': 1.0, 
    'd_control': 1.0
}

In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline

outputs = model(*(pairs), **(controls), gen=True)
mel = outputs[0][-1]
filter_0 = outputs[-2]
source_0 = outputs[-1]
audio = vocoder(mel)

plt.imshow(mel.squeeze(0).detach().cpu().numpy(), origin='lower', aspect='auto')
plt.show()

plt.imshow(filter_0.squeeze(0).detach().cpu().numpy(), origin='lower', aspect='auto')
plt.show()

plt.imshow(source_0.squeeze(0).detach().cpu().numpy(), origin='lower', aspect='auto')
plt.show()


sr = preprocess_config['preprocessing']['audio']['sampling_rate']
display(ipd.Audio(audio.squeeze().detach().cpu().numpy(), rate=sr, normalize=False, autoplay=False))